# DSM2 as a network

DSM2 is a network (graph theory) of junctions and reservoirs with connections of either channels, reservoir connections or gates. In this view of DSM2 the input tables that represent the hydrodynamic network can be represented as a mathematical graph.

This view allows some interesting questions to be asked and answered. 

E.g.
* What is the shortest path from one node to another? (if it exists)
* What nodes are downstream of this node?
* What nodes are upstream of this node?
* ... and many more graph related information

For illustrating this concept, this notebook shows how the tables can be easily converted to a graph using networkx package and answering some these interesting questions

In [ ]:
from pydsm.input import parser, network
import networkx as nx

## Read in input file
Open the hydro echo file and read it

In [ ]:
fname='../../tests/data/hydro_echo_historical_v82.inp'
with open(fname, 'r') as file:
    tables = parser.parse(file.read())

## Convert to a graph network 

The channel tables contain upnode and downnode connections. 

The reservoir_connection table contains the nodes to which the reservoirs connect. 

These can be used to construct a directed graph network

In [ ]:
c = tables['CHANNEL']
gc = nx.from_pandas_edgelist(c, source='UPNODE', target='DOWNNODE', edge_attr=list(
     c.columns), create_using=nx.MultiDiGraph)
rc = tables['RESERVOIR_CONNECTION']
grc1 = nx.from_pandas_edgelist(rc, source='RES_NAME', target='NODE', edge_attr=list(
     rc.columns), create_using=nx.MultiDiGraph)
grc2 = nx.from_pandas_edgelist(rc, source='NODE', target='RES_NAME', edge_attr=list(
     rc.columns), create_using=nx.MultiDiGraph)

## Drawing graph networks
Some examples of what these graphs construted from tables look like...

### Reservoir connections

In [ ]:
nx.draw(grc1, arrows=True, with_labels=True, pos=nx.spring_layout(grc1))

Combine the directed reservoir node connection graphs (one for incoming and other for outgoing)

In [ ]:
grc=nx.compose(grc1,grc2)
nx.draw(grc, arrows=True, with_labels=True, pos=nx.spring_layout(grc))

Combine reservoir connections and channel table for a complete graph

In [ ]:
gall=nx.compose(gc,grc)
nx.draw(gall, arrows=True, with_labels=True, pos=nx.spring_layout(gall))

Use the pydsm.input.network module to encapsulate the building in one method

In [ ]:
g=network.build_network(tables)

In [ ]:
nx.draw(g,arrows=True, with_labels=True, pos=nx.spring_layout(g))

## Utilizing graph information
You can leverage the graph theory to charaterize DSM2's flow network

In [ ]:
print('Is this graph strongly connected? :',nx.is_strongly_connected(gall))
print('Number of strongly connected components: ', nx.number_strongly_connected_components(gall))
print('Is this graph weakly connected? :',nx.is_weakly_connected(gall))
print('Number of weakly connected components: ', nx.number_weakly_connected_components(gall))
print('Weakly connected components: ', list(nx.weakly_connected_components(gall)))

### Show all the shortest paths between two nodes
E.g. in this case it is node 17 ( the San Joaquin River boundary) and the 361 (Martinez) 

In [ ]:
print(list(nx.all_shortest_paths(gall,17,361)))

### or between node and reservoir
or show the shortest paths between a reservoir and a node

In [ ]:
print(list(nx.all_shortest_paths(gall,'liberty',361)))

### Use holoviews visualization library for zoom and pan around functionality

In [ ]:
import hvplot.networkx as hvnx
hvnx.draw_spring(gall, labels='index', width=900, height=600)

### Graph Info

In [ ]:
print(nx.info(gall))

### Test that  all nodes connect to 361 (Boundary) 

In [ ]:
print('Limiting print to first 5 nodes')
for node in list(gall.nodes)[0:5]:
    print(list(nx.shortest_path(gall,node,361)))

### Test that not all nodes are connected to node 1

In [ ]:
try:
    for node in gall.nodes:
        print(list(nx.shortest_path(gall,node,1)))
    print('Test failed. Every node is connected to 1')
except Exception as e:
    print(e) 

In [ ]:
nx.is_directed_acyclic_graph(gall)

In [ ]:
nx.flow_hierarchy(gall)

In [ ]:
list(nx.neighbors(gall,1))

In [ ]:
list(nx.neighbors(gall,'mildred'))

In [ ]:
list(nx.neighbors(gall,'franks_tract'))